In [10]:
import tabula
import pandas as pd
import collections
import PySimpleGUI as sg
import os


colunas_tabela = ['Unnamed: 0', 'Processed\rChannel Descr.', 'Result\rSet ID',
       'Result\rID', 'SampleName', 'Vial', 'Inj.', 'Result\r#',
       '# of Results Stored', 'Altered', 'Manual\rResult', 'Manual\rCalib.',
       'Proc.\rMethod', 'Proc.Meth.\rID', 'Result Comments',
       'Result Set Comments', 'Number of\rSign Offs']
colunas_tabela_rename = ['Unnamed: 0', 'Processed ChannelDescr.', 'Result Set ID',
       'Result ID', 'SampleName', 'Vial', 'Inj.', 'Result #',
       '# of Results Stored', 'Altered', 'Manual Result', 'Manual Calib.',
       'Proc. Method', 'Proc.Meth.ID', 'Result Comments',
       'Result Set Comments', 'Number of Sign Offs']

colunas_tabela_erros = ['Unnamed: 0', 'Processed ChannelDescr.', 'Result Set ID',
       'Result ID', 'SampleName', 'Vial', 'Inj.', 'Result #',
       '# of Results Stored', 'Altered', 'Manual Result', 'Manual Calib.',
       'Proc. Method', 'Proc.Meth.ID', 'Result Comments',
       'Result Set Comments', 'Number of Sign Offs','Tipo de erro']

Erros_full = pd.DataFrame(columns =colunas_tabela_erros)

sg.theme('Material2')

menu_def = [      
                ['Ajuda', 'Instruções'], ]    
layout = [
        [sg.Menu(menu_def, tearoff=False)],
        [sg.Text('Insira os arquivos abaixo no formato PDF :',size=(35, 1))],
        [sg.Text('Arquivo PDF', size=(15, 1), auto_size_text=False, justification='right'),
        sg.Input(key= 'diretorio1'),sg.FileBrowse('Selecionar')],
        #[sg.Text('Arquivo txt 2', size=(15, 1), auto_size_text=False, justification='right'),
       # sg.Input(key= 'diretorio2'),sg.FileBrowse('Selecionar')],
        [sg.Text('_'  * 80)], 
        [sg.Button('Analisar arquivos'),sg.CloseButton('Fechar')]
]



window = sg.Window('Analisar arquivos', layout)

vent, values = window.read()
diretorio1 = values['diretorio1']
#diretorio2 = values['diretorio2']

while True:
    event, value = window.read()
    if event == 'Analisar arquivos':
        caminho_padrao = (diretorio1.split("/"))
        del caminho_padrao[-1]
        separator = '/'
        caminho_padrao = separator.join(caminho_padrao) + separator
        try:

            for pdfs in os.listdir(caminho_padrao):
                file = caminho_padrao + pdfs

                table = tabula.read_pdf(file,pages='all',lattice = True)
                df_columns_error = pd.DataFrame()
                df_columns_acerto = pd.DataFrame()
                
                for i in range(0,len(table)):
                    if len(table[i].columns) == 17:
                        if len(df_columns_acerto.columns) < 1 :
                            df_columns_acerto = table[i].copy()
                        else:
                            df_columns_acerto = pd.concat([df_columns_acerto,table[i]])
                    elif len(table[i].columns) >= 14:
                        if len(df_columns_acerto.columns) < 1 :
                            df_columns_acerto = table[i].copy()
                        else:
                            df_columns_acerto = pd.concat([df_columns_acerto,table[i]])
                    elif len(table[i].columns) <  17:
                        if len(df_columns_error.columns) < 1:
                            df_columns_error = table[i].copy()
                        else:
                            df_columns_error = pd.concat([df_columns_error,table[i]])
                if len(df_columns_error) == 0:
                    df = df_columns_acerto.copy()
                else:
                    df = pd.merge(df_columns_acerto,df_columns_error, on = 'Unnamed: 0')
                
                df = df[['Unnamed: 0', 'Processed\rChannel Descr.', 'Result\rSet ID',
               'Result\rID', 'SampleName', 'Vial', 'Inj.', 'Result\r#',
               '# of\rResults\rStored', 'Altered', 'Manual\rResult', 'Manual\rCalib.',
               'Proc.\rMethod', 'Proc.Meth.\rID', 'Result Comments',
               'Result Set Comments', 'Number of\rSign Offs']]
                df.columns = colunas_tabela_rename

                df.reset_index(drop=True,inplace =True)
                erro_result_set_id = pd.DataFrame(columns =colunas_tabela_erros)
                erro_filtered_df_inj = pd.DataFrame(columns =colunas_tabela_erros)
                erros_df_temp_false = pd.DataFrame(columns =colunas_tabela_erros)
                erro_manual_result = pd.DataFrame(columns =colunas_tabela_erros)
                erro_manual_calib = pd.DataFrame(columns =colunas_tabela_erros)
                erro_proc_method = pd.DataFrame(columns =colunas_tabela_erros)
                erro_proc_meth_id = pd.DataFrame(columns =colunas_tabela_erros)
                erro_number_of_sign_off = pd.DataFrame(columns =colunas_tabela_erros)

                # O número do result set ID deve ser igual para todas as linhas.

                if len(df[['Result Set ID']].drop_duplicates()) > 1:
                    erro_result_set_id = df[0:1]
                    erro_result_set_id.insert(loc = len(df.columns), column = 'Tipo de erro',value = 'O relatório possui mais de um Result Set ID.' )
                #erro_result_set_id

                # Verificar se há injeções maiores que um
                if int(df['Inj.'].max()) > 1 :
                    df_mask = df['Inj.'] > 1
                    erro_filtered_df_inj = df[df_mask]
                    erro_filtered_df_inj.insert(loc = len(df.columns), column = 'Tipo de erro',value = 'Inj. maiores que 1.' )

                #erro_filtered_df_inj

                #Result# - Se a coluna Result# for igual a um, a coluna Altered tem que ser igual a "No", se a coluna Result# 
                #for maior que um a coluna Altered tem que ser igual a "Yes"
                Resultado_verifica_result = []

                for verfica_zero_um in df.index:
                    if df.iloc[verfica_zero_um]['Result #'] == 1 and df.iloc[verfica_zero_um]['Altered'] == 'No' or df.iloc[verfica_zero_um]['Result #'] > 1 and df.iloc[verfica_zero_um]['Altered'] == 'Yes':
                        Resultado_verifica_result.append(True)
                    else:
                        Resultado_verifica_result.append(False)
                if False in Resultado_verifica_result:
                    df_temp = df.copy()
                    df_temp.insert(loc = len(df_temp.columns), column = 'Verifica_erro',value = Resultado_verifica_result )
                    mask_df_temp = df_temp['Verifica_erro'] == False
                    erros_df_temp_false = df_temp[mask_df_temp].drop(columns=['Verifica_erro'])
                    erros_df_temp_false.insert(loc = len(df.columns), column = 'Tipo de erro',value = 'As colunas Result# e Altered# estão fora do padrão.' )
                # erros_df_temp_false


                # Manual result  - O valor apresentado na coluna deve ser igual a "No".
                if 'Yes' in list(df['Manual Result']):
                    mask_manual_result = df['Manual Result'] == 'Yes'
                    erro_manual_result = df[mask_manual_result]
                    erro_manual_result.insert(loc = len(df.columns), column = 'Tipo de erro',value = 'Manual Result fora do padrão.')
                #erro_manual_result


                #Manual Calib - O valor apresentado na coluna deve ser igual a "No". 
                if 'Yes' in list(df['Manual Calib.']):
                    mask_manual_calib = df['Manual Calib.'] == 'Yes'
                    erro_manual_calib = df[mask_manual_calib]
                    erro_manual_calib.insert(loc = len(df.columns), column = 'Tipo de erro',value = 'Manual Calib fora do padrão.')
                #erro_manual_calib


                #Proc.Method - A regra será estruturada de acordo com o Sample Set (teor, id, diss,uc,sr): 
                #para os testes - Teor, id, uniformidade de conteúdo e de solução, (Sample set : "teor, id, diss,uc") 
                #não pode haver alteração na coluna, sempre será o mesmo dado para todas as linhas. Para os testes de substâncias 
                #relacionadas (Sample set  : "sr"), é aceitável tenha dados diferentes.
                teste_1 = [' TEOR',' ID',' DISS',' UC']
                teste_2 = [' SR']

                for teste_a in teste_1 :
                    if teste_a in df['Proc. Method'].values[0]:
                        if len(df[['Proc. Method']].drop_duplicates()) > 1:
                            erro_proc_method = df[0:1]
                            erro_proc_method.insert(loc = len(df.columns), column = 'Tipo de erro',value = 'O relatório possui mais de um Proc.Meth. ID .' )
                    elif teste_2[0] in df['Proc. Method'].values[0]:
                        pass
                #erro_proc_method

                #Proc.Meth. ID - A regra será estruturada de acordo com o Sample Set (teor, id, diss,uc,sr): para os testes -
                #Teor, id, uniformidade de conteúdo e de solução, (Sample set : "teor, id, diss,uc") não pode haver alteração na 
                #coluna, sempre será o mesmo dado para todas as linhas. Para os testes de substâncias relacionadas (Sample set  : "sr"),
                #é aceitável tenha dados diferentes.
                teste_1 = [' TEOR',' ID',' DISS',' UC']
                teste_2 = [' SR']
                for teste_b in teste_1 :
                    if teste_b in df['Proc. Method'].values[0]:
                        if len(df[['Proc.Meth.ID']].drop_duplicates()) > 1:
                            erro_proc_meth_id = df[0:1]
                            erro_proc_meth_id.insert(loc = len(df.columns), column = 'Tipo de erro',value = 'O relatório possui mais de um Proc.Meth. ID .' )
                    elif teste_2[0] in df['Proc. Method'].values[0]:
                        pass
                #erro_proc_meth_id

                pdList = [erro_result_set_id,erro_filtered_df_inj,erros_df_temp_false,erro_manual_result,erro_manual_calib,erro_proc_method,erro_proc_meth_id,erro_number_of_sign_off]
                df_erros = pd.concat(pdList)
                df_erros['Erro no arquivo'] = str(pdfs)
                try:
                    Erros_full.insert(loc= len(Erros_full.columns),column = 'Erro no arquivo', value = '')
                except:
                    pass
                Erros_full = pd.concat([Erros_full,df_erros])
            
            arquivo = caminho_padrao + 'Relatório de inconsistêcias.xlsx'
            Erros_full.to_excel(arquivo, index = False)
            sg.popup_notify('A análise dos arquivos foi finalizada !' + ' Foram \n encontradas ' +  str(len(Erros_full)) + ' inconsistências.')
            break
            window.close()
        except FileNotFoundError:
            sg.popup('Selecione pelo menos um diretório que contenha arquivos PDF !')
            break
            window.close()
            
    elif event == 'Instruções': 
        sg.popup('Para utilizar o programa de análise de PDF, siga as seguintes instruções :\n • Selecione a pasta aonde estão salvos apenas os arquivos PDF. \n • Clique em analisar arquivos.  \n • No final da análise será gerado um arquivo excel, contendo os erros presentes em cada relatório.')
        
    else:
        break
        window.close()